In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/startup_funding']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

## Hello Kagglers!!

### Today we will explore use Indian startup funding data to explore and gain knowledge about funding investment market. This will help us see wheather is India had heathly market for startup culture and their growth  

Data can be taken from (https://www.kaggle.com/sudalairajkumar/indian-startup-funding) or can be scraped from (https://www.trak.in) to get latest data

A. How does the funding ecosystem change with time?

B. Do cities play a major role in funding?

C. Which industries are favored by investors for funding?

D. Who are the important investors in the Indian Ecosystem?

E. How much funds does startups generally get in India?


## Importing required libraries

1. **Pandas:** For loading data from csv into dataframe
2. **datetime:** For converting date stored in string formate into datetime format to do analysis based on date, month, quarter, year
3. **re:** For extrarcting value from string using pattern
4. **matplotlib and seaborn:** For data visualization

In [1]:
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
from datetime import datetime
import re
# ALEX: remove plotting
# import matplotlib.pyplot as plt
# import seaborn as sns

# %matplotlib inline

# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# 1. Initial data exploration using shape, describe() and info()

file_path = "./input/startup_funding.scaled.csv"
data = pd.read_csv(file_path)
data.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [3]:
data.shape

(3044, 10)

In [4]:
data.columns

Index(['Sr No', 'Date dd/mm/yyyy', 'Startup Name', 'Industry Vertical',
       'SubVertical', 'City  Location', 'Investors Name', 'InvestmentnType',
       'Amount in USD', 'Remarks'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2084 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


In [6]:
# 2. The data types of data in few columns are not correct to do statistical analysis, hence it needs to be fixed using dtype, astype()

data.rename(columns={"Date dd/mm/yyyy": "Date", 'Startup Name': 'StartupName', 'Industry Vertical':'IndustryVertical',
       'SubVertical': 'SubVertical', 'City  Location': 'CityLocation', 'Investors Name': 'InvestorsName', 'InvestmentnType':'InvestmentnType',
       'Amount in USD':'AmountUSD', 'Remarks':'Remarks'}, inplace=True)
data.head()

,Sr No,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentnType,AmountUSD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [7]:
# 3. Few values 'Date' column are not in correcdt date format checking those rows
data[~data.Date.str.contains('(\d{2})[/](\d{2})[/](\d{4})')]

/tmp/ipykernel_115897/212197557.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[~data.Date.str.contains('(\d{2})[/](\d{2})[/](\d{4})')]


,Sr No,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentnType,AmountUSD,Remarks
192,193,05/072018,Loan Tap,Consumer Internet,Online Lending Platform For Salaried Professio...,Mumbai,"Shunwei Capital, Tuscan Ventures, Ashish Goenk...",Private Equity,"62,50,000",NaN
2571,2572,01/07/015,HandyHome,Hyperlocal Electronics repair Service,NaN,Mumbai,"Bessemer Ventures, Kae Capital",Seed Funding,"6,30,000",NaN
2606,2607,\\xc2\\xa010/7/2015,\\xc2\\xa0Infinity Assurance,\\xc2\\xa0Warranty Programs Service Administra...,NaN,\\xc2\\xa0New Delhi,\\xc2\\xa0Indian Angel Network,Seed Funding,"\\xc2\\xa0600,000",NaN
2775,2776,12/05.2015,Mobiefit,Mobile Fitness App,NaN,Goa,Prototyze,Seed Funding,NaN,NaN
2776,2777,12/05.2015,Plancess,Online Education Platform,NaN,Mumbai,"Rajendra Gogri,\\xc2\\xa0 Chandrakant Gogri",Seed Funding,"20,00,000",NaN
2831,2832,13/04.2015,The Porter,Logistics Tech,NaN,Mumbai,Kae Capital,Seed Funding,"5,00,000",NaN
3011,3012,15/01.2015,Wishberry,NaN,NaN,NaN,"Rajan Anandan, Shankar Mahadevan, Deep Kalra &...",Seed Funding,"6,50,000",NaN
3029,3030,22/01//2015,Corporate360,NaN,NaN,NaN,Group of Angel Investors,Seed Funding,"2,00,000",NaN


In [8]:
#incorrect_date = data[~data.Date.str.contains('(\d{2})[/](\d{2})[/](\d{4})')].index # [192, 2571, 2606, 2775, 2776, 2831, 3011, 3029]
# ALEX: make notebook run with input scaling
# data.loc[ 192, 'Date'] = '05/07/2018' 
# data.loc[2571, 'Date'] = '01/07/2015' 
# data.loc[2606, 'Date'] = '10/07/2015'
# data.loc[2775, 'Date'] = '12/05/2015' 
# data.loc[2776, 'Date'] = '12/05/2015'
# data.loc[2831, 'Date'] = '13/04/2015' 
# data.loc[3011, 'Date'] = '15/01/2015'
# data.loc[3029, 'Date'] = '22/01/2015'
incorrect_date = data[~data.Date.str.contains('(\d{2})[/](\d{2})[/](\d{4})')].index
replacements = ['05/07/2018', '01/07/2015', '10/07/2015', '12/05/2015',
                '12/05/2015', '13/04/2015', '15/01/2015', '22/01/2015']
for i, idx in enumerate(incorrect_date):
    data.loc[idx, 'Date'] = replacements[i % len(replacements)]

In [9]:
# 4. Extracting date, month, year from string values in 'Date' column

date_expand = data['Date'].str.extract(r'(\d{2})/?(\d{2})/?(\d{4})')
data['Year'] = date_expand[2]
data['Month'] = date_expand[1]
data['NewDate'] = date_expand[0]+'/'+date_expand[1]+'/'+date_expand[2]
data.head()

,Sr No,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentnType,AmountUSD,Remarks,Year,Month,NewDate
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN,2020,01,09/01/2020
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN,2020,01,13/01/2020
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN,2020,01,09/01/2020
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN,2020,01,02/01/2020
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN,2020,01,02/01/2020


In [10]:
data['Date'] = pd.to_datetime(data['Date'])#['Date']
data.head()

/tmp/ipykernel_115897/3494973059.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Date'] = pd.to_datetime(data['Date'])#['Date']


,Sr No,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentnType,AmountUSD,Remarks,Year,Month,NewDate
0,1,2020-09-01,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN,2020,01,09/01/2020
1,2,2020-01-13,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN,2020,01,13/01/2020
2,3,2020-09-01,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN,2020,01,09/01/2020
3,4,2020-02-01,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN,2020,01,02/01/2020
4,5,2020-02-01,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN,2020,01,02/01/2020


In [11]:
# 5. Converting datatype of values in 'AmountUSD' column from string to float. Marking Undisclosed values to 'nan' and then converting into float type

data.loc[data['AmountUSD'].isin(['undisclosed', 'unknown', 'Undisclosed']), 'AmountUSD'] = 'nan'

data['AmountUSD'] = data['AmountUSD'].astype(str)
data['NewAmountUSD'] = data['AmountUSD'].apply(lambda x : re.sub("[^0-9]", "", x))
data.loc[data['NewAmountUSD']=='', 'NewAmountUSD'] = 0 #'nan' # replace with average of funding provided that months 
data['NewAmountUSD'] = data['NewAmountUSD'].astype(float)
data.head()

,Sr No,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentnType,AmountUSD,Remarks,Year,Month,NewDate,NewAmountUSD
0,1,2020-09-01,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN,2020,01,09/01/2020,200000000.0
1,2,2020-01-13,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN,2020,01,13/01/2020,8048394.0
2,3,2020-09-01,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN,2020,01,09/01/2020,18358860.0
3,4,2020-02-01,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN,2020,01,02/01/2020,3000000.0
4,5,2020-02-01,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN,2020,01,02/01/2020,1800000.0


In [12]:
# 6. Cleaning column 'CityLocation'

data.loc[data['CityLocation'].isin(['\\\\xc2\\\\xa0Noida', '\\xc2\\xa0Noida']), 'CityLocation'] = 'Noida'
data.loc[data['CityLocation'].isin(['\\\\xc2\\\\xa0Bangalore', '\\xc2\\xa0Bangalore', 'Bangalore']), 'CityLocation'] = 'Bengaluru'
data.loc[data['CityLocation'].isin(['\\\\xc2\\\\xa0New Delhi', '\\xc2\\xa0New Delhi']), 'CityLocation'] = 'New Delhi'
data.loc[data['CityLocation'].isin(['\\\\xc2\\\\xa0Gurgaon', 'Gurugram']), 'CityLocation'] = 'Gurgaon'
data.loc[data['CityLocation'].isin(['\\\\xc2\\\\xa0Mumbai', '\\xc2\\xa0Mumbai']), 'CityLocation'] = 'Mumbai'
# len(data['CityLocation'].unique())

In [13]:
# 7. Cleanning column 'IndustryVertical'

data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0News Aggregator mobile app", 'IndustryVertical'] = 'News Aggregator mobile app'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Online Jewellery Store", 'IndustryVertical'] = 'Online Jewellery Store'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Fashion Info Aggregator App", 'IndustryVertical'] = 'Fashion Info Aggregator App'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Online Study Notes Marketplace", 'IndustryVertical'] = 'Online Study Notes Marketplace'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Warranty Programs Service Administration", 'IndustryVertical'] = 'Warranty Programs Service Administration'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Pre-School Chain", 'IndustryVertical'] = 'Pre-School Chain'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Premium Loyalty Rewards Point Management", 'IndustryVertical'] = 'Premium Loyalty Rewards Point Management'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Contact Center Software Platform", 'IndustryVertical'] = 'Contact Center Software Platform'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Casual Dining restaurant Chain", 'IndustryVertical'] = 'Casual Dining restaurant Chain'
data.loc[data['IndustryVertical'] == "\\\\xc2\\\\xa0Online Grocery Delivery", 'IndustryVertical'] = 'Online Grocery Delivery'
data.loc[data['IndustryVertical'] == "Online home d\\\\xc3\\\\xa9cor marketplace", 'IndustryVertical'] = 'Online home decor marketplace'
data.loc[data['IndustryVertical'].isin(["ECommerce", "E-Commerce", "E-commerce", "Ecommerce"]), 'IndustryVertical'] = 'eCommerce'
data.loc[data['IndustryVertical'].isin(["Fin-Tech"]), 'IndustryVertical'] = 'FinTech'

In [14]:
# 8. Cleanning column 'InvestorsName'

data.loc[data['InvestorsName'].isin(['Undisclosed investors', 'Undisclosed', 'undisclosed investors', 'Undisclosed Investor', 'Undisclosed investors']), 'InvestorsName'] = 'Undisclosed Investors'
data.loc[data['InvestorsName'] == "\\\\xc2\\\\xa0Tiger Global", 'InvestorsName'] = 'Tiger Global'
data.loc[data['InvestorsName'] == "\\\\xc2\\\\xa0IndianIdeas.com", 'InvestorsName'] = 'IndianIdeas'
data.loc[data['InvestorsName'] == "\\\\xc2\\\\xa0IvyCap Ventures, Accel Partners, Dragoneer Investment Group", 'InvestorsName'] = 'IvyCap Ventures, Accel Partners, Dragoneer Investment Group'
data.loc[data['InvestorsName'] == "\\\\xc2\\\\xa0Goldman Sachs", 'InvestorsName'] = 'Goldman Sachs'

## Starting EDA now!

In [15]:
startup_data = data[['Date', 'Year', 'Month', 'StartupName', 'IndustryVertical', 'SubVertical', 'CityLocation', 'InvestorsName', 'InvestmentnType', 'NewAmountUSD']]
startup_data['Date'] = pd.to_datetime(startup_data.Date)
startup_data.set_index('Date', inplace=True)
startup_data.head()

/tmp/ipykernel_115897/1555579611.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  startup_data['Date'] = pd.to_datetime(startup_data.Date)


,Year,Month,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentnType,NewAmountUSD
Date,,,,,,,,,
2020-09-01,2020,01,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,200000000.0
2020-01-13,2020,01,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,8048394.0
2020-09-01,2020,01,Mamaearth,eCommerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,18358860.0
2020-02-01,2020,01,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,3000000.0
2020-02-01,2020,01,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,1800000.0


# A. How does the funding ecosystem change with time?

In [16]:
funding_count_yr = pd.DataFrame(startup_data['Year'].value_counts())
funding_count_yr.rename(columns={"Year":"Number of Fundings"}, inplace=True)
funding_count_yr

,Number of Fundings
2016,993
2015,936
2017,687
2018,310
2019,111
2020,7


In [17]:
funding_count_qtr = pd.DataFrame(data=startup_data['Year'].resample('QS').count())
funding_count_qtr.rename(columns={'Year':'Number of Fundings(Qtr)'}, inplace=True)
# ALEX: make notebook run with input scaling
# funding_count_qtr['QtrMonth'] = ['2015-1', '2015-4', '2015-7', '2015-10', '2016-1', '2016-4', '2016-7', '2016-10', '2017-1', '2017-4', '2017-7', '2017-10', '2018-1', '2018-4', '2018-7', '2018-10', '2019-1', '2019-4', '2019-7', '2019-10', '2020-1', '2020-4', '2020-7', '2020-10']
funding_count_qtr['QtrMonth'] = ['2015-1', '2015-4', '2015-7',
                                 '2015-10', '2016-1', '2016-4',
                                 '2016-7', '2016-10', '2017-1',
                                 '2017-4', '2017-7', '2017-10',
                                 '2018-1', '2018-4', '2018-7',
                                 '2018-10', '2019-1', '2019-4',
                                 '2019-7', '2019-10', '2020-1',
                                 '2020-4', '2020-7', '2020-10'][:len(funding_count_qtr)]
funding_count_qtr.head()

,Number of Fundings(Qtr),QtrMonth
Date,,
2015-01-01,209,2015-1
2015-04-01,215,2015-4
2015-07-01,275,2015-7
2015-10-01,237,2015-10
2016-01-01,251,2016-1


In [18]:
funding_total_yr = pd.DataFrame(startup_data.groupby(by=['Year'])['NewAmountUSD'].sum())
funding_total_yr.rename(columns={"NewAmountUSD":"Total Funding(USD-Bn)"}, inplace=True)
funding_total_yr = funding_total_yr.sort_values(by='Total Funding(USD-Bn)', ascending=False)
funding_total_yr

,Total Funding(USD-Bn)
Year,
2015,1.691302e+10
2019,1.195203e+10
2017,1.042931e+10
2018,5.122368e+09
2016,3.828089e+09
2020,3.902073e+08


In [19]:
funding_total_qtr = pd.DataFrame(data=startup_data['NewAmountUSD'].resample('QS').sum())
funding_total_qtr.rename(columns={'NewAmountUSD':'Total Fundings(Qtr USD-Bn)'}, inplace=True)
# ALEX: make notebook run with input scaling
# funding_total_qtr['QtrMonth'] = ['2015-1', '2015-4', '2015-7', '2015-10', '2016-1', '2016-4', '2016-7', '2016-10', '2017-1', '2017-4', '2017-7', '2017-10', '2018-1', '2018-4', '2018-7', '2018-10', '2019-1', '2019-4', '2019-7', '2019-10', '2020-1', '2020-4', '2020-7', '2020-10']
funding_total_qtr['QtrMonth'] = ['2015-1', '2015-4', '2015-7',
                                 '2015-10', '2016-1', '2016-4',
                                 '2016-7', '2016-10', '2017-1',
                                 '2017-4', '2017-7', '2017-10',
                                 '2018-1', '2018-4', '2018-7',
                                 '2018-10', '2019-1', '2019-4',
                                 '2019-7', '2019-10', '2020-1',
                                 '2020-4', '2020-7', '2020-10'][:len(funding_total_qtr)]
funding_total_qtr.head()

,Total Fundings(Qtr USD-Bn),QtrMonth
Date,,
2015-01-01,1.972822e+09,2015-1
2015-04-01,1.334455e+09,2015-4
2015-07-01,9.944259e+09,2015-7
2015-10-01,3.661487e+09,2015-10
2016-01-01,1.104478e+09,2016-1


In [20]:
# ALEX: remove plotting
# fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18,8))

# sns.barplot(x=funding_count_yr.index, y=funding_count_yr['Number of Fundings'], data=funding_count_yr, ax=axes[0,0], orient='v')
# sns.barplot(x=funding_count_qtr.index, y=funding_count_qtr['Number of Fundings(Qtr)'], data=funding_count_qtr, ax=axes[0,1], orient='v').set_xticklabels(rotation=90, labels=funding_total_qtr['QtrMonth'])

# sns.barplot(x=funding_total_yr.index, y=funding_total_yr['Total Funding(USD-Bn)'], data=funding_total_yr, ax=axes[1,0], orient='v')
# sns.barplot(x=funding_total_qtr.index, y=funding_total_qtr['Total Fundings(Qtr USD-Bn)'], data=funding_total_qtr, ax=axes[1,1], orient='v').set_xticklabels(rotation=90, labels=funding_total_qtr['QtrMonth'])

# fig.tight_layout(pad=3)
# plt.show()
_ = funding_count_yr.index
_ = funding_count_yr['Number of Fundings']
_ = funding_count_qtr.index
_ = funding_count_qtr['Number of Fundings(Qtr)']
_ = funding_total_qtr['QtrMonth']
_ = funding_total_yr.index
_ = funding_total_yr['Total Funding(USD-Bn)']
_ = funding_total_qtr.index
_ = funding_total_qtr['Total Fundings(Qtr USD-Bn)']
_ = funding_total_qtr['QtrMonth']

## Exploring number and total fundings received by indian startups:

### **From above exploration it seems that the ***number of fundings*** made in 2016 was highest(initial quarters in 2016) followed by 2015 but it gradually started decreasing.**

-> Lot of startups got the funding in the year 2016 and 2015, this might be due to *Startup India, an initiative of the Government of India* declared in August 2015. 
-> Giving a boost to new entrepreneur getting funds and transforming ideas into reality. Year 2015 and 2016 has spread positivity in indian startup investor market!!

### **Getting highest number of fundings in 2016 does not mean the total funds raised was highest in 2016 (which was only 0.38 Billion).** 

-> The Total amount invested in startups was highest in 2015(majorly 3rd Quarter of 2015 when policy was declared by the Gov.) with 1.6 billion USD followed by in 2019 ~ 1.2 billion USD.

### **It seems number of fundings slowed down after 2016 (one of the reason can be Demonatization which affected many industries and markets). **

-> We have now entered in 2020 with lowest funding raised compared to past data. There seems some issue with domestic investors, why fundings are going down?

### The best is yet to come for the investors and the startups looking for fundings.. Hope upcoming days and policies are positive to help them to build amazing projects..

## B. Do cities play a major role in funding?
We all know that the Bangluru is startup hub of India, lets see if the data is supporting the same or an other city is emerging in this race?

In [21]:
fundings_count_city = pd.DataFrame(startup_data['CityLocation'].value_counts().sort_values(ascending=False)[:10])
fundings_count_city.rename(columns={'CityLocation':'Number of Fundings by City'}, inplace=True)
fundings_count_city.head()

,Number of Fundings by City
Bengaluru,842
Mumbai,568
New Delhi,424
Gurgaon,341
Pune,105


In [22]:
funding_total_city = pd.DataFrame(startup_data.groupby('CityLocation')['NewAmountUSD'].sum()).sort_values(by="NewAmountUSD", ascending=False)[:15]
funding_total_city.rename(columns={'NewAmountUSD':'Total Funding by City(USD-Bn)'}, inplace=True)
funding_total_city.head()

,Total Funding by City(USD-Bn)
CityLocation,
Bengaluru,2.198655e+10
Mumbai,6.940535e+09
New Delhi,5.048417e+09
Gurgaon,4.581021e+09
Noida,3.282864e+09


In [23]:
# ALEX: remove plotting
# fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(25, 10))

# sns.barplot(x=fundings_count_city.index, y=fundings_count_city['Number of Fundings by City'], data=fundings_count_city, ax=axes[0])
# sns.barplot(x=funding_total_city.index, y=funding_total_city['Total Funding by City(USD-Bn)'], data=funding_total_city, ax=axes[1]).set_xticklabels(rotation=90, labels=funding_total_city.index)

# fig.tight_layout(pad=0.5)
# plt.show()
_ = fundings_count_city.index
_ = fundings_count_city['Number of Fundings by City']
_ = funding_total_city.index
_ = funding_total_city['Total Funding by City(USD-Bn)']
_ = funding_total_city.index

## We were correct, Bangaluru it is.. the Sillicon Valley of India. 

-> There is a saying that anyone who comes to Mumbai with willpower will find his/her way to earn and live here. It seems true for statups in Bangaluru as well, the city is providing good atmospheare and is encoraging budding ideas to come to reality. 

-> Bangaluru has major share of its own by helping Indian economy i.e. by providing employment. 

-> Total fundings received by Bangaluru is highest at about 2.19 Billion and It drastically drops with Mumbai and NCR regions(Delhi, Gurgaon and Noida) followed by Chennai, Pune, Hydrabad. 


## **This explains why is their salary difference in Bangaluru and other cities!! :) Bangaluru take me.. mold me.. teach me.. let me exlore your awsomeness..** :P


## C. Which industries are favored by investors for funding?

In [24]:
fundings_count_industry = pd.DataFrame(startup_data['IndustryVertical'].value_counts().sort_values(ascending=False))[:15]
fundings_count_industry.rename(columns={'IndustryVertical':'Number of Fundings by Industry'}, inplace=True)
fundings_count_industry.head()

,Number of Fundings by Industry
Consumer Internet,941
Technology,478
eCommerce,296
Healthcare,70
Finance,62


In [25]:
funding_total_industry = pd.DataFrame(startup_data.groupby('IndustryVertical')['NewAmountUSD'].sum()).sort_values(by="NewAmountUSD", ascending=False)[:15]
funding_total_industry.rename(columns={'NewAmountUSD':'Total Funding by Industry(USD-Bn)'}, inplace=True)
funding_total_industry.head()

,Total Funding by Industry(USD-Bn)
IndustryVertical,
eCommerce,8.742763e+09
Consumer Internet,6.253084e+09
Transportation,3.916632e+09
Technology,2.229708e+09
News Aggregator mobile app,2.020000e+09


In [26]:
# ALEX: remove plotting
# fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(25, 15))

# sns.barplot(x=fundings_count_industry.index, y=fundings_count_industry['Number of Fundings by Industry'], data=fundings_count_industry, ax=axes[0]).set_xticklabels(rotation=90, labels=fundings_count_industry.index)
# sns.barplot(x=funding_total_industry.index, y=funding_total_industry['Total Funding by Industry(USD-Bn)'], data=funding_total_industry, ax=axes[1]).set_xticklabels(rotation=90, labels=funding_total_industry.index)

# fig.tight_layout(pad=1)
# #plt.xticks(rotation=90)
# plt.show()
_ = fundings_count_industry.index
_ = fundings_count_industry['Number of Fundings by Industry']
_ = fundings_count_industry.index
_ = funding_total_industry.index
_ = funding_total_industry['Total Funding by Industry(USD-Bn)']
_ = funding_total_industry.index

## 941 statups have got the fundings for **Consumer Internet Industry**, that seems perfectly fine as the users cunsumption growing, and we are part of it. 

-> Contribution in research and projects in Technology, eCommerce and Helthcare has boosted fundings and new automations are coming in to help the cosumers.

-> The total amount of fundings received for eCommerce is 0.87 billion and followed by Consumer Internet(0.6 billion), Trasportation(0.39 billion) and Technology(0.22 Billion)

### Many new startups are coming in the market to serve us like online Jewellery, news aggregation apps, grocery delivery, online pharmacy, educational content provider, electric scooter manufactoring. We are definatly growing and catching up with western world.

## D. Who are the important investors in the Indian Ecosystem?

In [27]:
funding_count_investor = pd.DataFrame(startup_data['InvestorsName'].value_counts()).sort_values(by='InvestorsName', ascending=False)[:10]
funding_count_investor.rename(columns={'InvestorsName': 'Number of Investments by Investor'}, inplace=True)
funding_count_investor.drop(funding_count_investor[funding_count_investor.index == 'Undisclosed Investors'].index, inplace=True)
funding_count_investor.head()

,Number of Investments by Investor
Ratan Tata,25
Indian Angel Network,23
Kalaari Capital,16
Group of Angel Investors,15
Sequoia Capital,15


In [28]:
funding_total_investor = pd.DataFrame(startup_data.groupby(['InvestorsName'])['NewAmountUSD'].sum()).sort_values(by="NewAmountUSD", ascending=False)[:15]
funding_total_investor.rename(columns={'NewAmountUSD':'Total Funding by Investor(USD-Bn)'}, inplace=True)
funding_total_investor.head()

,Total Funding by Investor(USD-Bn)
InvestorsName,
Westbridge Capital,3.900000e+09
Softbank,2.500000e+09
Tiger Global,2.130500e+09
Goldman Sachs,2.035000e+09
IndianIdeas,2.019350e+09


In [29]:
# ALEX: remove plotting
# fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(25, 15))

# sns.barplot(x=funding_count_investor.index, y=funding_count_investor['Number of Investments by Investor'], data=funding_count_investor, ax=axes[0]).set_xticklabels(rotation=90, labels=funding_count_investor.index)
# sns.barplot(x=funding_total_investor.index, y=funding_total_investor['Total Funding by Investor(USD-Bn)'], data=funding_total_investor, ax=axes[1]).set_xticklabels(rotation=90, labels=funding_total_investor.index)

# fig.tight_layout(pad=1)
# plt.show()
_ = funding_count_investor.index
_ = funding_count_investor['Number of Investments by Investor']
_ = funding_count_investor.index
_ = funding_total_investor.index
_ = funding_total_investor['Total Funding by Investor(USD-Bn)']
_ = funding_total_investor.index

In [30]:
startup_data[startup_data['InvestorsName'].isin(['Westbridge Capital', 'Softbank'])]

,Year,Month,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentnType,NewAmountUSD
Date,,,,,,,,,
2019-08-27,2019,08,Rapido Bike Taxi,Transportation,Bike Taxi,Bengaluru,Westbridge Capital,Series B,3.900000e+09
2017-11-08,2017,08,Flipkart,eCommerce,Online Marketplace,Bengaluru,Softbank,Private Equity,2.500000e+09


## A one-man venture capital fund is funding Indian startups giving lot of credibility and future. 

-> Except number of "Undisclosed Investors" the second rank comes to **Sir Ratan Tata**, Number of startups he has funded are 25 and increasing. This is followed by Indian Angel Network(23), Kalaari Capital(16) etc.

-> Total amount invested 0.39 billion USD by Westbridge Capital	is the highest in 'Rapido Bike Taxi' and Softbank has invested 0.29 billion USD in Flipkart

## E.How much funds does startups generally get in India?

In [31]:
funding_count_company = pd.DataFrame(startup_data['StartupName'].value_counts()).sort_values(by='StartupName', ascending=False)[:15]
funding_count_company.rename(columns={'StartupName': 'Number of Investments by Investor'}, inplace=True)
funding_count_company.head()

,Number of Investments by Investor
Ola Cabs,8
Swiggy,8
Paytm,7
Nykaa,6
NoBroker,6


In [32]:
funding_total_company = pd.DataFrame(startup_data.groupby('StartupName')['NewAmountUSD'].sum()).sort_values(by='NewAmountUSD', ascending=False)[:15]
funding_total_company.rename(columns={'NewAmountUSD': "Total amount Raised by Startup (USD-Bn)"}, inplace=True)
funding_total_company.head()

,Total amount Raised by Startup (USD-Bn)
StartupName,
Flipkart,4.059700e+09
Rapido Bike Taxi,3.900000e+09
Paytm,3.148950e+09
\\xc2\\xa0News in shorts,2.020000e+09
\\xc2\\xa0Loylty Rewards,2.019350e+09


In [33]:
# ALEX: remove plotting
# fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(25, 15))

# sns.barplot(x=funding_count_company.index, y=funding_count_company['Number of Investments by Investor'], data=funding_count_company, ax=axes[0]).set_xticklabels(rotation=90, labels=funding_count_company.index)
# sns.barplot(x=funding_total_company.index, y=funding_total_company['Total amount Raised by Startup (USD-Bn)'], data=funding_total_company, ax=axes[1]).set_xticklabels(rotation=90, labels=funding_total_company.index)

# fig.tight_layout(pad=1)
# plt.show()
_ = funding_count_company.index
_ = funding_count_company['Number of Investments by Investor']
_ = funding_count_company.index
_ = funding_total_company.index
_ = funding_total_company['Total amount Raised by Startup (USD-Bn)']
_ = funding_total_company.index

### Swiggy, Ola Cabs, Paytm, Nykaa etc has got lot of fundings these years. This due to their constant innovation in surving cunsumer with AI.

-> Total funding received by Flipkart is highest 0.4 billion USD followed by Rapido Bike Taxi with 0.39 billion USD.

In [34]:
# ALEX: remove plotting
# plt.figure(figsize=(25,8))
# sns.distplot(startup_data.loc[startup_data['NewAmountUSD']<=10000000.0, 'NewAmountUSD'])
# plt.show()
_ = startup_data.loc[startup_data['NewAmountUSD']<=10000000.0, 'NewAmountUSD']

In [35]:
funding_average = startup_data['NewAmountUSD'].mean()
funding_meadian = startup_data['NewAmountUSD'].median()
print(funding_average, funding_meadian)

15977341.477003941 500000.0


### Average fundings received is 15 million USD and most of the companies get 5 lac. Indian startup market is helping new innovations to come to live and encouraging entrepreneurs. 

-> Ending my EDA on this data, got to learn so much about startupd and investor market in India, hope to see more growth in investers.

-> Please give me feedback and suggestions which will help me grow more.

-> I am data enthusiast and building my career in Data Science. 
Please reach out to me on LinkedIn (https://www.linkedin.com/in/charulata-shelar-9726984a)

see you soon...